# Libaries

In [1]:
pip install bitsandbytes transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
pip install -U bitsandbytes

In [3]:
pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00


In [4]:
import torch
import time
import torch.quantization
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator

# Load Pre-trained Model and Dataset

In [5]:
# Load pre-trained model and tokenizer
model_name = 'textattack/bert-base-uncased-imdb'    # The model 'textattack/bert-base-uncased-imdb' is a BERT-based model, fine-tuned for sentiment classification on the IMDB dataset.
original_model = AutoModelForSequenceClassification.from_pretrained(model_name)   # # Load the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)   # # Load the corresponding tokenizer for the pre-trained model.

# Load imdb data
dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

# Subsample and Shuffle the IMDB Test Dataset

In [6]:
# Reduce the test set size to 2000 samples for faster experimentation
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(2000))

# Tokenize the input text using the pre-trained tokenizer.
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    # Extracts the text from the dataset
    # Truncate sequences longer than max_length
    # Pad shorter sequences to max_length
    # Define the maximum token length

# Tokenize dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)
# Specify that the dataset should be formatted as PyTorch tensors.
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

# Function to Measure Model Performance

    Measures key performance metrics of a given model, including:
    - Model storage size in megabytes (MB).
    - Inference speed (average latency per sample in seconds).
    - Model accuracy on a given dataset.

    Args:
        model (torch.nn.Module): The neural network model to evaluate.
        dataset (torch.utils.data.Dataset): The dataset for evaluation.

    Returns:
        dict: A dictionary containing:
            - "model_size_mb" (float): Model size in MB.
            - "avg_latency_seconds" (float): Average inference time per sample.
            - "accuracy" (float): Model classification accuracy.

In [7]:
# Measure model storage size, inference speed, and accuracy
def measure_performance(model, dataset):
    model.eval()

    # Compute model storage size (MB)
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())  # Calculate the total size of all model parameters (weights)
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())    # Calculate the total size of model buffers
    model_size_mb = (param_size + buffer_size) / (1024 ** 2)                    # Convert the total model size from bytes to megabytes (MB)

    # Ensure correct DataLoader format
    dataloader = DataLoader(dataset, batch_size=8, collate_fn=default_data_collator)  # Process 8 samples per batch

    # Initialize metrics for inference time, accuracy calculation
    total_time, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for batch in dataloader:
            # Move input tensors to the model's device (GPU)
            inputs = {
                "input_ids": batch["input_ids"].to(model.device),  # Match model's device
                "attention_mask": batch["attention_mask"].to(model.device),
            }

            # Move labels to the same device as the model
            labels = batch["labels"].to(model.device)

            # Measure inference time per batch
            start_time = time.time()
            outputs = model(**inputs)               # Forward pass through the model
            total_time += time.time() - start_time  # Accumulate total time

            # Obtain predicted class labels
            predictions = outputs.logits.argmax(dim=-1)

            # Count correct predictions
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    avg_latency = total_time / total   # Average inference time per sample
    accuracy = correct / total         # Compute accuracy

    # Return performance metrics as a dictionary
    return {"model_size_mb": model_size_mb, "avg_latency_seconds": avg_latency, "accuracy": accuracy}

# Function to Compare the Performance of Different Models

    Compares the original model with a set of optimized models by evaluating:
    - Model size reduction (percentage decrease in storage size).
    - Inference speed improvement (speedup factor).
    - Accuracy retention (accuracy ratio compared to the original model).

    Args:
        original_model (torch.nn.Module): The baseline (original) model.
        optimized_models (dict): A dictionary of optimized models, where:
            - Key (str): The name of the optimized model (e.g., "quantized", "pruned").
            - Value (torch.nn.Module): The optimized model instance.
        dataset (torch.utils.data.Dataset): The dataset for model evaluation.

    Returns:
        dict: A dictionary containing:
            - "original": Performance metrics of the original model.
            - Each optimized model's name as a key with its corresponding metrics:
                - Model size in MB
                - Average inference latency per sample
                - Accuracy
                - Size reduction percentage
                - Speedup factor
                - Accuracy retention ratio

In [8]:
def compare_models(original_model, optimized_models, dataset):
    # Initialize a dictionary to store the results for all models
    results = {}

    print("Evaluating original model...")
    original_metrics = measure_performance(original_model, dataset)

    # Store original model performance results
    results["original"] = original_metrics

    for name, model in optimized_models.items():
        print(f"Evaluating {name}...")
        metrics = measure_performance(model, dataset)  # Measure the performance of the optimized model

        # Compute relative improvements compared to the original model
        rel_metrics = {
            "size_reduction": 1 - (metrics["model_size_mb"] / original_metrics["model_size_mb"]),
            "speedup": original_metrics["avg_latency_seconds"] / metrics["avg_latency_seconds"],
            "accuracy_retention": metrics["accuracy"] / original_metrics["accuracy"]
        }

        # Store metrics in the results dictionary
        results[name] = {**metrics, **rel_metrics}

    return results

# Define an Optimized 8-bit Quantization Configuration

In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")

# Quantization reduces the model's memory footprint by storing weights in lower precision (8-bit)
quantization_config_8_bit = BitsAndBytesConfig(
    load_in_8bit=True,                      # Enables loading the model in 8-bit precision
    llm_int8_threshold=1.0,                 # Controls outlier handling (1.0 means no special handling)
    llm_int8_enable_fp32_cpu_offload=True,  # Offloads large FP32 operations to CPU for efficiency
    llm_int8_skip_modules=["classifier", "self_attention", "ffn", "layernorm"]  # Skips quantization for certain modules to prevent accuracy degradation
)

# Load the optimized 8-bit quantized model
quantized_model_8_bit = AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-imdb",
    quantization_config=quantization_config_8_bit
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


# Define an Optimized 4-bit Quantization Configuration

In [10]:
# This configuration enables 4-bit quantization, which further reduces model size
quantization_config_4_bit_set1 = BitsAndBytesConfig(
    load_in_4bit=True,                      # Use 4-bit quantization for better compression
    bnb_4bit_compute_dtype=torch.bfloat16,  # Set computation to "bfloat16"
    bnb_4bit_quant_type="nf4",              # Use "nf4" (Normal Float 4)
    bnb_4bit_use_double_quant=True          # Enable Double Quantization for more compression
)

# Load the optimized 4-bit quantized model
quantized_model_4_bit_set1 = AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-imdb",
    quantization_config=quantization_config_4_bit_set1
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


# Define another Optimized 4-bit Quantization Configuration

In [11]:
# Define another Optimized 4-bit Quantization Configuration
quantization_config_4_bit_set2 = BitsAndBytesConfig(
    load_in_4bit=True,                      # Use 4-bit quantization for better compression
    bnb_4bit_compute_dtype=torch.float16,   # Set computation to "float16"
    bnb_4bit_quant_type="nf4",              # Use "nf4" (Normal Float 4)
    bnb_4bit_use_double_quant=False         # Disable Double Quantization
)

# Load the optimized 4-bit quantized model
quantized_model_4_bit_set2 = AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-imdb",
    quantization_config=quantization_config_4_bit_set2
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


# Model Performance Evaluation

**Run 10 times**

## 1.

In [12]:
# Check if a CUDA-compatible GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the original model for comparison
original_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb").to(device)

# Ensure optimized_models is a dictionary
optimized_models = {"quantized_model_8_bit": quantized_model_8_bit,
                    "quantized_model_4_bit_set1": quantized_model_4_bit_set1,
                    "quantized_model_4_bit_set2": quantized_model_4_bit_set2}

# Run model comparison
results1 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results1.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Using device: cuda
Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0013
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0080
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1630
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0032
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.4023
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0021
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.6160
  accuracy_retention: 0.9994


## 2.

In [13]:
# Run model comparison
results2 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results2.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0077
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1455
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0033
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3443
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0022
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5081
  accuracy_retention: 0.9994


## 3.

In [14]:
# Run model comparison
results3 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results3.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0078
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1388
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0034
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3131
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0020
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5341
  accuracy_retention: 0.9994


## 4.

In [15]:
# Run model comparison
results4 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results4.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0082
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1369
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0033
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3447
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0022
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5128
  accuracy_retention: 0.9994


## 5.

In [16]:
# Run model comparison
results5 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results5.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0079
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1364
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0035
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3071
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0020
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5497
  accuracy_retention: 0.9994


## 6.

In [17]:
# Run model comparison
results6 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results6.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0012
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0078
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1484
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0033
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3557
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0020
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5810
  accuracy_retention: 0.9994


## 7.

In [18]:
# Run model comparison
results7 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results7.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0079
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1357
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0033
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3243
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0020
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5356
  accuracy_retention: 0.9994


## 8.

In [19]:
# Run model comparison
results8 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results8.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0078
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1442
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0034
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3354
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0019
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5817
  accuracy_retention: 0.9994


## 9.

In [20]:
# Run model comparison
results9 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results9.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0080
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1323
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0032
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3296
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0022
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.4786
  accuracy_retention: 0.9994


## 10.

In [21]:
# Run model comparison
results10 = compare_models(original_model, optimized_models, encoded_dataset["test"])

# Print results
for model_name, metrics in results10.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

Evaluating original model...
Evaluating quantized_model_8_bit...
Evaluating quantized_model_4_bit_set1...
Evaluating quantized_model_4_bit_set2...

Model: original
  model_size_mb: 417.6553
  avg_latency_seconds: 0.0011
  accuracy: 0.8920

Model: quantized_model_8_bit
  model_size_mb: 127.2690
  avg_latency_seconds: 0.0081
  accuracy: 0.8875
  size_reduction: 0.6953
  speedup: 0.1329
  accuracy_retention: 0.9950

Model: quantized_model_4_bit_set1
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0033
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.3212
  accuracy_retention: 0.9994

Model: quantized_model_4_bit_set2
  model_size_mb: 86.4878
  avg_latency_seconds: 0.0019
  accuracy: 0.8915
  size_reduction: 0.7929
  speedup: 0.5551
  accuracy_retention: 0.9994


## Get Average

In [22]:
import pandas as pd

# Sample structure assuming results1 to results10 exist
results_list = [results1, results2, results3, results4, results5, results6, results7, results8, results9, results10]

# Convert results into a structured format
data = []
for result in results_list:
    for model_name, metrics in result.items():
        row = {'Model': model_name}
        row.update(metrics)
        data.append(row)

# Create a DataFrame
df = pd.DataFrame(data)

# Compute mean values for each model
mean_results = df.groupby('Model').mean(numeric_only=True).reset_index()

# Display the results
from IPython.display import display
display(mean_results)

,Model,model_size_mb,avg_latency_seconds,accuracy,size_reduction,speedup,accuracy_retention
0,original,417.655281,0.001119,0.8920,NaN,NaN,NaN
1,quantized_model_4_bit_set1,86.487797,0.003318,0.8915,0.792921,0.337765,0.999439
2,quantized_model_4_bit_set2,86.487797,0.002057,0.8915,0.792921,0.545268,0.999439
3,quantized_model_8_bit,127.269047,0.007918,0.8875,0.695277,0.141412,0.994955
